# Property

> Fill in a module description here

In [ ]:
# | default_exp core.property

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# | export
from dataclasses import dataclass
from abc import ABC, abstractmethod

from exex.imports import *
from exex.core.event import *
from exex.core.unit import *
from exex.utils import camel_to_snake

### Data

In [ ]:
# | export
class Object:
    pass

In [ ]:
# | export
class PropertyData(dict):
    pass

In [ ]:
c = PropertyData({"0": {"mass": 2, "object": Object()}})

In [ ]:
c["0"]

{'mass': 2, 'object': <__main__.Object>}

In [ ]:
c["1"] = {"mass": 2.1, "object": "XXX"}

### Proprety

In [ ]:
# | export
@docs
class BaseProperty:
    def __init__(self, cmp):  # chemical substance
        self.is_constant: bool = False
        self.compound = cmp
        self.cmp = cmp
        self._data = PropertyData()

        self._connections = []
        self.laws = dict()
        self.func_changed = Event()

    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @classmethod
    @property
    def snake_name(cls) -> str:  # return the snake style name
        return camel_to_snake(cls.__name__)

    def expr(self, t: int):  # time
        return self.symbol(t)

    def __call__(self, t: int, eval: bool = False, **kwargs):  # time
        self.t = t
        self.kwargs = {**kwargs, "eval": eval}

        expr = self.expr(t)

        if eval == True:
            return self.eval(expr, t)
        else:
            return expr

    def add_law(self, law):
        if not law in self.laws:
            self.laws[camel_to_snake(law.__class__.__name__)] = law

    _docs = dict(cls_doc="Property", add_law="", expr="Symbolic expression")

In [ ]:
# | export
@patch
def symbol(self: BaseProperty, t):  # symbolic expression of the property
    """Rewrite this method if you want to modify"""
    return smp.symbols(f"{self.abbrv}_{self.cmp.snake_name}-{t}", real=True)

In [ ]:
# | export
@patch
def set_val(self: BaseProperty, val: str, t: int):
    self._data[t] = {"val": val}

In [ ]:
# | export
@patch
def get_val(self: BaseProperty, t: int):  # time
    if self.is_constant is True:
        return self.compute()
    else:
        return self._data[t]["val"] if t in self._data else self.symbol(t)

In [ ]:
# | export
@patch
def eval(self: BaseProperty, expr, t: int):  # express  # time
    return expr.xreplace({expr: self.get_val(t=t)})

In [ ]:
# | export
@patch
def is_empty(self: BaseProperty, t):
    return type(self.get_val(t))
    # return True if isinstance(type(self.get_val(t)), sympy.core.symbol.Symbol) else False

In [ ]:
# | export
@docs
class Property(BaseProperty):
    _docs = dict(cls_doc="Property that varies in time")

#### Constant Property

In [ ]:
# | export
@docs
class ConstantProperty(BaseProperty):
    @abstractmethod
    def compute(self):
        pass

    _docs = dict(
        cls_doc="Property that invariant in time", compute="Calculate the value"
    )

##### Property

In [ ]:
# | export
class Property(BaseProperty):
    pass

In [ ]:
# | export
class PropertyObservable(Property):
    pass

##### Molar Mass

Molar Mass is

In [ ]:
# # | export
# class Mass(Property):
#     def __init__(self, compound):
#         super().__init__(compound)
#         self.abbrv = "m"
#         self.unit = Unit.MASS

In [ ]:
# # | export
# class MolarMass(ConstantProperty):
#     def __init__(self, compound):
#         self.abbrv = "M"
#         self.unit = Unit.MOLAR_MASS
#         super().__init__(compound)
#         self.is_constant = True

#     def compute(self):
#         mass = 0
#         for element in self.compound.elements:
#             mass += element.AtomicMass

#         return mass * self.unit

In [ ]:
# # | export
# class Mole(Property):
#     def __init__(self, compound):
#         super().__init__(compound)
#         self.abbrv = "n"
#         self.unit = Unit.MOLE

In [ ]:
# # | export
# class Pressure(Property):
#     def __init__(self, compound):
#         super().__init__(compound)
#         self.abbrv = "P"
#         self.unit = Unit.PRESSURE

In [ ]:
# # | export
# class Volume(Property):
#     def __init__(self, compound):
#         super().__init__(compound)
#         self.abbrv = "V"
#         self.unit = Unit.VOLUME

In [ ]:
# # | export
# class Temperature(Property):
#     def __init__(self, compound):
#         super().__init__(compound)
#         self.abbrv = "T"
#         self.unit = Unit.TEMPERATURE